# Basic Network Sourcing example
Uses one production, two intermediate and multiple consumption locations.
We're moving one product "Beer" measured in the dimension "weight"
How are movements in the network costed?
- We have two lane rates between our main production center and the two warehouses
- and a distribution "Cost Model" between the sources (proudction + 2 x warehouses) and consumption nodes
- Lane rates between Production and Intermediate nodes are costed on a cost per km basis.
- Cost models to distribute the quantities further is also based on a (more expensive) cost per km.
- It's typical that high utilisation vehicles move between warehouses (and typically larger vehicles, achieving a lower cost per km / cost per ton)
- And that smaller vehicles handle the secondary distribution (at a higher cost per ton)

In [ ]:
import pandas
df = pandas.read_csv('../sample_data/publist_large.csv')
# we'll just use the first 100 nodes for a free-tier key
df = df.head(100) # free tier key, model too large
print(df.head())

In [ ]:
exec(open('apiHelper.py').read()) # import some helper classes which we've written for you.

api = apiHelper(modelType="ns3-tbfvuwtge2iq")

Lets instantiate a model container so that we can build out a model


In [ ]:
sr = ns3_tbfvuwtge2iq_pb2.SolveRequest()
sr.geometryOutput = 1  # set the geometry output to aggregate
sr.solveType = 0       # optimise solve request.
m = sr.model
print(sr)

Lets load the model helper which builds some common objects used in this model

In [ ]:
exec(open('ns3-model-helper.py').read()) # import some modelling helpers


### Model outline
* this model is going to define distance, time and weight (the dimensions).
* we'll create a single production node, two warehouse nodes, and the balance as demand (or consumption) nodes.
* we'll add lane rates between the production nodes and the warehouses, and costmodels for the warehouses (which will automatically connect with the demand nodes)
* This model will use a single product ("Beer") to illustrate a minimal model definition.

In [ ]:
m.dimensions.CopyFrom(make_distance_time_user_dimensions('weight'))
m.dimensions # we have time, distance and weight

We're going to split the data into three node components. 
1. A product node where the demand is -1
2. Warehouse nodes where the demand is -2
3. All other nodes where the demand > 0

In [ ]:

productionNodes = df[df['demand'] == -1]
warehouseNodes =  df[df['demand'] == -2]
demandNodes = df[df['demand'] > 0]
print(productionNodes)
print('-------------')
print(warehouseNodes)
print('-------------')
print(demandNodes.head())


Lets now convert all these data rows into nodes which we can use in the model.

In [ ]:
p_nodes = make_nodes(productionNodes)
w_nodes = make_nodes(warehouseNodes)
d_nodes = make_nodes(demandNodes)
print(p_nodes)
print('-------------')
print(w_nodes)

Lets assume we can go factory-direct or through a warehouse!

In [ ]:
sources = list(df[df['demand'] < 0]['id'])
sources

Now we're going to define which nodes require a certain amount of each dimension (consumption nodes), followed by the nodes where volumes can be produced (production)

In [ ]:
for i, o in enumerate(d_nodes):
    pf = ns3_tbfvuwtge2iq_pb2.Node.ProductFlow()
    pf.productId = 'Beer'
    # each demand node must have the quantity demand[i] delivered, so the range here
    # is actually [demand[i], demand[i]]. 
    # Not meeting this range incurs a large penalty cost.
    di = demandNodes.iloc[i]['demand']
    pf.dimensionRanges.append(make_dimension_range("weight", di, di))
    del d_nodes[i].consumption[:] #in case you run this twice
    d_nodes[i].consumption.append(pf)
    del d_nodes[i].allowableSources[:] # in case you run this twice
    d_nodes[i].allowableSources.extend(sources) # all sources are allowable.
    
print(d_nodes[0])

In [ ]:
for i, o in enumerate(p_nodes):
    pf = ns3_tbfvuwtge2iq_pb2.Node.ProductFlow()
    pf.productId = 'Beer'
    # the production node has no limit on the amount that can be produced. 
    # so we can simply set the upper bound to the sum of all demand, i.e. [0, sum(demands)]
    # this way we know that the facility can produce enough to satisfy all the demand
    pf.dimensionRanges.append(make_dimension_range("weight", 0,  sum(demandNodes['demand'])))
    del p_nodes[i].production[:] #in case you run this twice
    p_nodes[i].production.append(pf)
    
print(p_nodes[0])


Now that the node definitions are complete, we can add them all to the model

In [ ]:
del m.nodes[:]
m.nodes.extend(p_nodes)
m.nodes.extend(w_nodes)
m.nodes.extend(d_nodes)
print(len(m.nodes))


We're going to define some lane rates between the main nodes in the network.
1. Guiness Storehouse -> Limerick
2. and Guinnes Storehouse -> Galway

each costed at 0.1 monetary units per km.

In [ ]:
del m.laneRates[:]
m.laneRates.append(make_lane_rate_distance(sources[0], sources[1], 0.1))
m.laneRates.append(make_lane_rate_distance(sources[0], sources[2], 0.1))
print(m.laneRates)

Lets define the product group we're using

In [ ]:
del m.productGroups[:]
m.productGroups.append(make_single_product_group("Beer", "weight"))
print(m.productGroups)

Next up we're going to define the cost models for the secondary distribution from each of the sources to the delivery footprint.

In [ ]:
del m.costModels[:]
for index, i in enumerate(sources):
    m.costModels.append(make_cost_model_distance(i, 0.2))

print(m.costModels)

### Sending the model
We now have a complete model specified so we can submit it to the api.

In [ ]:
reqId = api.Post(sr)
print(reqId) # if you want to see what the guid looks like.

### Retrieve the model response
We can use the api-helper to grab the solution response using the request-id provided by the post.

In [ ]:
sol = api.Get(reqId)

print(sol.objective) # This is the cost of the solution returned by the api.


# Interpreting the response
The network sourcing model returns item which can be easily tabulated. For this reason, we've wrapped a couple of common functions in the apiHelper.py file which can be used to tabulate the response. 
The geometries are returned in a special format in this model which are indexed by their common overlapping sections (so that the response payload is smaller). 

There are three main tables, the assignments (i.e. which lane rates or cost models are selected), the node flow, and node product flow (which indicate the flow _over_ a node, either in the aggregate or by product).

In [ ]:
tab = tabulate(sr, sol)
print ("--------- assignments --------- ")
print (tab['assignments'].head())
print ("\n--------- node flows --------- ")
print (tab['nodeFlows'].head())
print ("\n--------- node product flows --------- ")
print (tab['nodeProductFlows'].head())

In [ ]:
tab['routes'].head() # and these are the geometries between each of the locations.

### Visualising the response

Lets go ahead and visualise the assignments.

In [ ]:
from ipyleaflet import Map, Polyline, Circle, LayerGroup

# add some locations to the map (just black dots should be ok)
locs = list()
for index, p in tab['nodes'].iterrows():
    circle = Circle()
    circle.location = (p['y'], p['x'])
    circle.radius = 10
    circle.color = "black"
    locs.append(circle)

center = [df['latitude'][1],df['longitude'][1]] # just use the first point as the center (i.e. the guiness storehouse)
m = Map(scroll_wheel_zoom=True, center=center, zoom=6)
for i, gs in enumerate(tab['routes']['geometry']):
    m.add_layer(Polyline(locations=gs,color='blue', fill=False))

m.add_layer(LayerGroup(layers=(locs)))
m

### What's next?

There are intermediate and advanced examples which build on this particular model to illustrate how to use fixed cost triggers and other features to encapsulate more real-world costs.

Alternatively, if you're feeling adequately prepared, why not try with your with data?
